# Benchmarking a TF-TF network against literature-reported TF-TF interactions
Use this notebook to benchmark the filtered networks produced from `filter_assertions.ipynb`.

### Setup

In [ ]:
import pandas as pd
import numpy as np
import glob
import tqdm
import random
import statistics 
from scipy import stats
import os
from matplotlib import pyplot as plt
from collections import defaultdict

Set a random seed to ensure reproducibility.

In [ ]:
random.seed(314)

Add a subdirectory to hold the libraries used for benchmarking and one to store the results. 

In [ ]:
lib_dir = './raw_data/benchmarking_libraries'
output_dir = './benchmarking_results'

if not os.path.exists(lib_dir):
    os.mkdir(lib_dir)
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

Define a function to upload the benchmarking files as source-target pairs. This function finds all such pairs in a GMT-formatted benchmarking file, where each GMT term is a TF target. Only targets found in the set of high-rank TFs are included, since these are the targets with which we constructed the original network.

In [ ]:
def reformat(gene):
    if gene == "BORCS8MEF2B":
        return "BORCS8-MEF2B"
    if "NKX" in gene:
        return gene[0:-1] + "-" + gene[-1]
    if gene == "C11ORF95":
        return "C11orf95"
    else:
        return gene
    
def find_library_edges(lib, tflib):

    '''
    creates a dictionary of source-target pairs in a GMT-formatted benchmarking file
    each term is the source, targets are TFs in that term's set
    input: a library and set of valid transcription factors
    return: a dataframe of (source,target) edges and edge counts
    '''

    edges = {}

    # for each line in that library file, 
    with open(lib, 'r') as f:
        for line in f:
            # extract the gmt term and gene set
            items = line.strip().split("\t")
            term = items[0]

            # check the term/set formatting
            if items[1] != "":
                targets = items[1:]
            else:
                targets = items[2:]

            # extract the source TF from the term name 
            parts = term.strip().split(" ")

            if len(parts) == 1:
                source = term.split("_")[0].upper()
            else:
                source = parts[0].upper()
            
            source = reformat(source)
            # check if source is in transcript library -- if not, we know it's a TF, add it
            if source not in tflib and source.upper() != "C11ORF95":
                tflib.append(source)

            # add the edge to the list
            for target in targets:
                t = reformat(target.upper())
                if t in tflib and t.upper() != "C11ORF95":
                    if (source,t) in edges.keys():
                        edges[(source,t)] += 1
                    else:
                        edges[(source,t)] = 1

    # form dataframe and add counts
    edgeidx = pd.MultiIndex.from_tuples(edges.keys(), names = ['source', 'target'])
    edgedf = pd.DataFrame(index = edgeidx, columns = ['count'])

    for (s,t), count in edges.items():
        edgedf.loc[(s,t)] = count
    
    return edgedf

This function compares a specified network edge list against all three benchmarking libraries. The significance of the overlap between the two networks is determined by randomly shuffling the input network 100 times, then performing a z-test using the expected overlap and standard deviation. The results are saved to a CSV file for easy inspection.

In [ ]:
def benchmark_edges(network_type, libraries):

    '''
    Compares an input network against a set of benchmarking libraries
    input: the type of network being compared and the benchmarking libraries being used 
    return: a dataframe containing the results organized by benchmarking library
    output: .csv file of the results for a single network
    '''

    # open network edge file and extract source,target tuples
    network_edge_file = f'./filtered_edge_list/large/{network_type}/edge_list_filtered.csv'
    network_edges = pd.read_csv(network_edge_file, usecols=[0, 1])
    n_ntwrk_edges = len(network_edges)

    # store results individually for each library 
    results = {}
    trial_undirected = {}
    trial_directed = {}
    obs_overlap_undirected = {}
    obs_overlap_directed = {}

    # for each benchmarking library, generate random trials and store the results in the dictionaries above
    for blib, library_counts in libraries.items():

        # calculate the DIRECTED overlap ratio 
        library_edgelist = library_counts.index.to_frame(index=False)
        common_edges = pd.merge(library_edgelist, network_edges, how='inner')

        overlap = len(common_edges) / n_ntwrk_edges

        # calculate the undirected overlap ratio
        network_edges_undirected = pd.DataFrame([sorted(edge) for edge in network_edges.to_numpy()], columns = ['source', 'target'])
        library_edges_undirected = pd.DataFrame([sorted(edge) for edge in library_edgelist.to_numpy()], columns = ['source', 'target'])
        common_edges_undirected = pd.merge(library_edges_undirected, network_edges_undirected, how='inner')

        undirected_overlap = len(common_edges_undirected) / n_ntwrk_edges

        # generate a random set of edges using the original benchmark library and calculate the significance of the observed overlap
        N_TRIALS = 100
        trial_results = {
            'directed': np.zeros(N_TRIALS),
            'undirected': np.zeros(N_TRIALS)
        }

        ntwrk_source_labels = list(set(network_edges['source']))
        ntwrk_target_labels = list(set(network_edges['target']))

        for i in tqdm.tqdm(range(N_TRIALS)):
            # generate a new network using the weighted node distribution from the library network
            rand_sources = random.choices(ntwrk_source_labels, k=n_ntwrk_edges)
            rand_targets = random.choices(ntwrk_target_labels, k=n_ntwrk_edges)

            # calculate the overlap and save 
            random_network = pd.DataFrame(zip(rand_sources, rand_targets), columns = ['source', 'target'])
            random_network_undirected = pd.DataFrame([sorted(edge) for edge in random_network.to_numpy()], columns = ['source', 'target'])
            
            common_edges_trial = pd.merge(library_edgelist, random_network, how='inner')
            common_edges_undir_trial = pd.merge(library_edges_undirected, random_network_undirected, how='inner')

            trial_results['directed'][i] = len(common_edges_trial) / n_ntwrk_edges
            trial_results['undirected'][i] = len(common_edges_undir_trial) / n_ntwrk_edges
        
        # calculate statistics of random trials
        expected_overlap = statistics.mean(trial_results['directed'])
        overlap_stdev = statistics.stdev(trial_results['directed'])

        expected_overlap_undir = statistics.mean(trial_results['undirected'])
        overlap_stdev_undir = statistics.stdev(trial_results['undirected'])

        z_score = (overlap - expected_overlap) / overlap_stdev
        undir_z_score = (undirected_overlap - expected_overlap_undir) / overlap_stdev_undir
        
        # only calculate p-value for cases where there is more than expected overlap
        p_value = stats.norm.sf(z_score)
        undir_p = stats.norm.sf(undir_z_score)

        benchmarklib = (blib.split("/")[-1]).split(".")[0].replace("_", " ")

        # save results
        trial_undirected[benchmarklib] = trial_results['undirected']
        trial_directed[benchmarklib] = trial_results['directed']
        obs_overlap_directed[benchmarklib] = overlap
        obs_overlap_undirected[benchmarklib] = undirected_overlap
        
        results[benchmarklib] = {
            'library network size': len(library_counts),
            'input network size': n_ntwrk_edges,
            'observed overlap, directed': overlap,
            'observed overlap, undirected': undirected_overlap,
            'expected overlap, directed': expected_overlap,
            'expected overlap, undirected': expected_overlap_undir,
            'stdev, directed': overlap_stdev,
            'stdev, undirected': overlap_stdev_undir,
            'p-value directed': p_value,
            'p-value, undirected': undir_p
            }

    undirected = dict(sorted(trial_undirected.items(), key=lambda item: statistics.mean(item[1])))
    directed = dict(sorted(trial_directed.items(), key=lambda item: statistics.mean(item[1])))
    resultsdf = pd.DataFrame(results)

    resultsdf.to_csv(f"{output_dir}/{network_type}.csv")
    
    return {
        'directed': directed, 
        'undirected': undirected, 
        'overlap-dir': obs_overlap_directed, 
        'overlap-undir': obs_overlap_undirected
        }

### Benchmarking

Upload the full list of high-rank TFs using `mean_ranks_matrix.csv`  (produced by `construct_edge_list.ipynb`). This will be our library of TFs. 

In [ ]:
# load only the first column of the mean ranks matrix, which contains all of the TFs identified by ChEA3
path_to_mean_ranks = './edge_constructing_files/mean_ranks_matrix.csv'
tf_library = pd.read_csv(path_to_mean_ranks, usecols=[0])
tf_library = tf_library.iloc[:, 0].tolist()
tf_library = [reformat(tf.upper()) for tf in tf_library]

We'll use three libraries to perform the benchmarking. 1. - 2. are Enrichr libraries, and 3. - 8. are the six ChEA3 primary libraries used in `construct_edge_list.ipynb`:
1. TRANSFAC and JASPAR PWMs
2. TRRUST Transcription Factors 2019
3. ARCHS4 Coexpression
4. ENCODE ChIP-seq
5. Enrichr Queries
6. Literature ChIP-seq
7. ReMap ChIP-seq
8. GTEx coexpression

Download the Enrichr libraries by navigating here and clicking on their names: https://maayanlab.cloud/Enrichr/#libraries.\
Move all six libraries into `raw_data/benchmarking_libraries`, copying the six ChEA3 libraries from `raw_data/chea3libs`. 

Parse the benchmarking library names from their file paths. 

In [ ]:
# specify order of files to make processing easier 
enrichrfiles = glob.glob(f'{lib_dir}/*.txt')
chea3files = glob.glob(f'{lib_dir}/*.gmt')

allfiles = enrichrfiles + chea3files

# format filenames
filenames = [(file.split("/")[-1]).split(".")[0].replace("_", " ") for file in allfiles]

Calculate the benchmarking results for all three network types. 

In [ ]:
libraries = {file:find_library_edges(file, tf_library) for file in allfiles}

In [ ]:
ntypes = ['signature_shuffling', 'node_weighted']
network_results = {ntype:benchmark_edges(ntype, libraries) for ntype in ntypes}

# Benchmarking figures

In [ ]:
import matplotlib.ticker as ticker
import seaborn as sns

In [ ]:
# initate dict to store results - will be keyed by library
undirected_reform = {lib:{'network':[],'values':[]} for lib in filenames}
directed_reform = {lib:{'network':[],'values':[]} for lib in filenames}
obsv_directed = {lib:{'network':[],'values':[]} for lib in filenames}
obsv_undirected = {lib:{'network':[],'values':[]} for lib in filenames}

ntrials = 100
for ntwrk, libs in network_results.items():
    for lib,value1 in libs['undirected'].items():
        undirected_reform[lib]['network'].append(ntwrk)
        undirected_reform[lib]['values'].append(value1.tolist())

    for lib, value2 in libs['directed'].items():
        directed_reform[lib]['network'].append(ntwrk)
        directed_reform[lib]['values'].append(value2.tolist())

    for lib,value3 in libs['overlap-dir'].items():
        obsv_directed[lib]['network'].append(ntwrk)
        obsv_directed[lib]['values'].append(value3)

    for lib, value4 in libs['overlap-undir'].items():
        obsv_undirected[lib]['network'].append(ntwrk)
        obsv_undirected[lib]['values'].append(value4)

In [ ]:
for lib in filenames:
    obsv_dir = pd.DataFrame.from_dict(obsv_directed[lib]).set_index('network').reset_index()
    obsv_undir = pd.DataFrame.from_dict(obsv_undirected[lib]).set_index('network').reset_index()
    results = pd.DataFrame.from_dict(directed_reform[lib]).set_index('network')['values'].explode().reset_index()
    results2 = pd.DataFrame.from_dict(undirected_reform[lib]).set_index('network')['values'].explode().reset_index()

    for df in [obsv_dir, obsv_undir, results, results2]:
        df.replace(to_replace={'signature_shuffling':'TSS-filtered GRN','node_weighted':'ND-filtered GRN'}, inplace=True)
    f, ax = plt.subplots(nrows=2, ncols=1, figsize=(7,8), dpi=300)
    print(lib)

    plt.subplots_adjust(hspace=0.4)

    sns.set_theme(font_scale=2)

    sns.boxplot(results, ax = ax[0], x='values', y='network', hue='network', palette='vlag', width=0.5)
    sns.stripplot(results, ax = ax[0], x='values', y='network', size=4, color=".5")
    sns.stripplot(obsv_dir, ax = ax[0], x='values', y='network', size=8, color="black")
    ax[0].set_xlabel('Overlap, directed')
    ax[0].set_ylabel(None)
    ax[0].set_title(lib)

    sns.set_theme(font_scale=2)
    sns.boxplot(results2, ax = ax[1], x='values', y='network', hue='network', palette='vlag', width=0.5)
    sns.stripplot(results2, ax = ax[1], x='values', y='network', size=4, color=".5")
    sns.stripplot(obsv_undir, ax = ax[1], x='values', y='network', size=8, color="black")
    ax[1].set_xlabel("Overlap, undirected")
    ax[1].set_ylabel(None)

    plt.savefig(f"../figures/{lib}.svg", format="svg")